# **特徵縮放與學習率（多變量）**

## 目標
在本實驗中，你將：
- 使用在前一個實驗中建立的多變量相關函式
- 在具有多個特徵的資料集上執行梯度下降（Gradient Descent）
- 探索「學習率 α」對梯度下降的影響
- 利用 z-score 正規化進行特徵縮放，以提升梯度下降的效能

<br>

## 工具
你將會使用上一個實驗中撰寫的函式，以及 `matplotlib` 與 `NumPy` 等套件。

In [ ]:
# region 資料載入
import sys, os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None

def import_data_from_github():
    import urllib.request, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_utils_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/utils"
        else:
            return Path.cwd() / "utils"

    def get_data_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/data"
        else:
            return Path.cwd() / "data"

    def get_images_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return f"/content/images"
        else:
            return Path.cwd() / "images"

    env = detect_env()
    UTILS_DIR = get_utils_dir(env)
    DATA_DIR = get_data_dir(env)
    IMG_DIR = get_images_dir(env)

    REPO_DIR = "Machine-Learning-Lab"

    #shutil.rmtree(UTILS_DIR, ignore_errors=True)
    os.makedirs(UTILS_DIR, exist_ok=True)
    #shutil.rmtree(DATA_DIR, ignore_errors=True)
    os.makedirs(DATA_DIR, exist_ok=True)
    os.makedirs(IMG_DIR, exist_ok=True)

    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    urllib.request.urlretrieve(f"{BASE}/utils/lab_utils_multi.py", f"{UTILS_DIR}/lab_utils_multi.py")
    urllib.request.urlretrieve(f"{BASE}/utils/lab_utils_common.py", f"{UTILS_DIR}/lab_utils_common.py")
    urllib.request.urlretrieve(f"{BASE}/utils/deeplearning.mplstyle", f"{UTILS_DIR}/deeplearning.mplstyle")
    urllib.request.urlretrieve(f"{BASE}/data/houses.txt", f"{DATA_DIR}/houses.txt")

    images_list = ["shortRun.PNG", "longRun.PNG", "scale.PNG"]
    for image in images_list:
        urllib.request.urlretrieve(f"{BASE}/lab/teacher/Regression/images/{image}", f"{IMG_DIR}/{image}")

repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()
    
os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

from utils.lab_utils_multi import  load_house_data, run_gradient_descent 
from utils.lab_utils_multi import  norm_plot, plt_equal_scale, plot_cost_i_w
from utils.lab_utils_common import dlc
np.set_printoptions(precision=2)

plt.style.use('utils/deeplearning.mplstyle')
print("✅ 匯入模組及設定繪圖樣式")
#endregion 資料載入

<br>

## 符號說明（Notation）

<div align="center">

| 一般符號 | 說明 | Python（若適用） |
| ------------| ------------------------------------------------------------|----------------|
| $a$ | 標量（scalar），非粗體 |
| $\mathbf{a}$ | 向量（vector），粗體小寫 |
| $\mathbf{A}$ | 矩陣（matrix），粗體大寫 |
| **Regression（迴歸）** |  |  |
| $\mathbf{X}$ | 訓練樣本矩陣 | `X_train` |   
| $\mathbf{y}$ | 訓練樣本目標值 | `y_train` |
| $\mathbf{x}^{(i)}$, $y^{(i)}$ | 第 $i$ 筆訓練樣本 | `X[i]`, `y[i]`|
| m | 訓練樣本數 | `m`|
| n | 每筆樣本的特徵數 | `n`|
| $\mathbf{w}$ | 參數：權重（weights） | `w` |
| $b$ | 參數：偏差（bias） | `b` |     
| $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ | 在樣本 $\mathbf{x}^{(i)}$ 上，由參數 $\mathbf{w},b$ 所計算出的模型輸出：$f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)}+b$ | `f_wb` | 
|$\frac{\partial J(\mathbf{w},b)}{\partial w_j}$| 成本函數對參數 $w_j$ 的偏導數（梯度） |`dj_dw[j]`| 
|$\frac{\partial J(\mathbf{w},b)}{\partial b}$| 成本函數對參數 $b$ 的偏導數（梯度）| `dj_db`|

</div>

<br>

# 問題說明（Problem Statement）

與先前的實驗一樣，我們會以房價預測作為例子。訓練資料集中包含許多筆樣本，每筆樣本有 4 個特徵（面積、臥室數、樓層數、屋齡），如下面的表格所示。請注意，在本實驗中，面積（Size）使用的是「平方英尺（sqft）」為單位，而在較早的實驗中使用的是「以 1000 平方英尺為單位」。此外，這個資料集比前一個實驗使用的還要大。

我們希望利用這些特徵建立一個線性迴歸模型，之後就能用它來預測其他房屋的價格──例如：一棟 1200 平方英尺、3 間臥室、1 層樓、屋齡 40 年的房子。

## 資料集（Dataset）

<div align="center">

| 面積（Size, sqft） | 臥室數（Number of Bedrooms） | 樓層數（Number of floors） | 屋齡（Age of Home） | 價格（千美元, Price） |
| -------------------|------------------------------|---------------------------|---------------------|------------------------|
| 952                | 2                            | 1                         | 65                  | 271.5                  |
| 1244               | 3                            | 2                         | 64                  | 232                    |
| 1947               | 3                            | 2                         | 17                  | 509.8                  |
| ...                | ...                          | ...                       | ...                 | ...                    |

</div>


In [ ]:
# load the dataset
X_train, y_train = load_house_data()
X_features = ['size(sqft)','bedrooms','floors','age']

讓我們將每個特徵與房價繪圖，來看看資料集與各個特徵的關係。

In [ ]:
fig,ax=plt.subplots(1, 4, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("Price (1000's)")
plt.show()

將每一個特徵與目標值（房價）畫在一起，可以幫助我們判斷哪些特徵對價格的影響較大。上面的圖中可以看到：房屋面積愈大，價格通常愈高；臥室數與樓層數看起來對價格的影響並不那麼明顯；而較新的房子通常比老舊房子有更高的價格。

<a name="toc_15456_5"></a>
## 多變量梯度下降（Gradient Descent With Multiple Variables）
下面是你在上一個「多變量梯度下降」實驗中推導出的公式：

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline\;
& w_j := w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j = 0..n-1}\newline
&b\ \ := b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

其中，n 是特徵的數量，參數 $w_j$ 與 $b$ 會同時被更新，而

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3}
\end{align}
$$
* m 是資料集中訓練樣本的數量

    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ 是模型在樣本 $\mathbf{x}^{(i)}$ 上的預測值，而 $y^{(i)}$ 則是對應的目標值


<br>

## 學習率（Learning Rate）

在課程中，我們討論了與設定學習率 $\alpha$ 相關的一些問題。學習率會控制每次更新參數時的步伐大小（請參考上面的式子 (1)），而且是**所有參數共用同一個值**。  

接下來，我們就在這個資料集上執行梯度下降，並嘗試幾種不同的 $\alpha$ 設定。

### $\alpha$ = 9.9e-7（學習率較大時的行為）

In [ ]:
#set alpha to 9.9e-7
_, _, hist = run_gradient_descent(X_train, y_train, 10, alpha = 9.9e-7)

看起來學習率設定得太高了，解並沒有收斂，成本函數的值反而在「上升」而不是下降。讓我們把結果畫出來看看：

In [ ]:
plot_cost_i_w(X_train, y_train, hist)

右邊的圖顯示其中一個參數 $w_0$ 的變化情形。在每一次迭代中，它都「超過」最佳值太多，導致成本不但沒有往最小值靠近，反而持續上升。請注意，這個圖並不是完整的真實情況，因為實際上每一次更新有 4 個參數同時在改變，而圖中只顯示 $w_0$，其他參數則固定在較溫和的數值上。在這張圖以及後面幾張圖中，你可能會注意到藍色與橘色的曲線有些微偏差，這是因為它只是高維情況的一個投影。


### $\alpha$ = 9e-7
讓我們把學習率再調小一點，看看會發生什麼事。

In [ ]:
#set alpha to 9e-7
_,_,hist = run_gradient_descent(X_train, y_train, 10, alpha = 9e-7)

在整個訓練過程中，成本都在持續下降，顯示這個 alpha 值並不算太大。

In [ ]:
plot_cost_i_w(X_train, y_train, hist)

左圖中可以看到，成本確實如預期般地在下降。右圖中則可以看到 $w_0$ 仍然在最小值附近來回震盪，不過每一次迭代時，成本仍然持續下降而不是上升。注意上方圖中，`dj_dw[0]` 在每次迭代時都會改變符號，代表 `w[0]` 持續在最佳值的兩側跳動。
這個 alpha 值最終會收斂，你可以嘗試改變迭代次數來觀察它的行為。

### $\alpha$ = 1e-7
我們再把 $\alpha$ 調得更小一些，來看看結果如何。

In [ ]:
#set alpha to 1e-7
_,_,hist = run_gradient_descent(X_train, y_train, 10, alpha = 1e-7)

在整個訓練過程中，成本持續下降，顯示這個 $\alpha$ 值同樣不算太大。

In [ ]:
plot_cost_i_w(X_train,y_train,hist)

左圖中可以看到，成本如預期持續下降。右圖則顯示 $w_0$ 在沒有明顯震盪的情況下逐漸逼近最小值，且在整個訓練過程中，`dj_w0` 都維持為負值。這樣的設定同樣會收斂。

<br>

## 特徵縮放（Feature Scaling）

課程中說明了，將資料集中的各個特徵重新縮放到「相近的數值範圍」是非常重要的。

如果你想更深入了解為什麼要這麼做，可以展開下方的「Details」區塊；若暫時不需要細節，往下的內容會示範如何實作特徵縮放。

<details>
<summary>
    <font size='3', color='darkgreen'><b>Details</b></font>
</summary>

讓我們再次回頭看看 $\alpha$ = 9e-7 的情況。這個值非常接近在不發散的前提下所能設定的最大值。下面這張圖是只跑前幾次迭代的「短程訓練」結果：

<figure>
    <img src="./images/shortRun.PNG" style="width:1200px;" >
</figure>

可以看到，雖然成本正在下降，但很明顯由於梯度較大，$w_0$ 比其他參數前進得快得多。

下一張圖顯示的是使用 $\alpha$ = 9e-7、跑了非常久（需要好幾個小時）的「長程訓練」結果：

<figure>
    <img src="./images/longRun.PNG" style="width:1200px;" >
</figure>
    
從圖中可以看到，成本在最初快速下降之後，後續的下降速度就變得非常緩慢。請留意 `w0` 與 `w1`,`w2`,`w3` 之間的差異，以及 `dj_dw0` 與 `dj_dw1-3` 的差異：`w0` 很快就接近最後的值，而 `dj_dw0` 也迅速下降到很小的數值，代表 `w0` 已經接近最終解；相比之下，其他參數的變化就慢得多。

為什麼會這樣？有什麼可以改進的嗎？請看下面這張圖：
<figure>
    <center> <img src="./images/scale.PNG"   ></center>
</figure>   

這張圖說明了為什麼各個 $w$ 的更新速度會差這麼多：
- $\alpha$ 是所有參數（$w$ 和 $b$）共用的。
- 對 $w$ 而言，更新時會把共同的誤差項乘上對應的特徵值（而 $b$ 則不會乘上特徵）。
- 各個特徵的數值尺度差距很大，使得某些特徵對應的參數更新得比其他參數快很多。例如，$w_0$ 乘上的特徵是 `size(sqft)`，通常大於 1000，而 $w_1$ 乘上的特徵是 `number of bedrooms`，通常只在 2–4 之間。 
    
解決方法就是「特徵縮放（Feature Scaling）」。

課程中介紹了三種不同的技巧：

- **Feature scaling（特徵縮放）**：基本做法是將每個「非負」特徵除以其最大值，或更一般地，使用 (x-min)/(max-min) 來根據最小值與最大值重新縮放。兩種方式都可以把特徵縮放到大約 -1 到 1 的範圍；前者適用於全為正值的特徵、實作簡單，後者則較通用，適用於任意特徵。

- **Mean normalization（平均值正規化）**：$x_i := \dfrac{x_i - \mu_i}{\max - \min}$ 

- **Z-score normalization（z 分數正規化）**：這是我們接下來要實作與探討的重點。


### z-score 正規化（z-score normalization）
在進行 z-score 正規化之後，所有特徵都會有平均值 0，且標準差為 1。

要實作 z-score 正規化，可以依照下列公式調整輸入值：
$$x^{(i)}_j = \dfrac{x^{(i)}_j - \mu_j}{\sigma_j} \tag{4}$$ 
其中，$j$ 表示在矩陣 $\mathbf{X}$ 中選取的特徵或欄位；$\mu_j$ 是第 $j$ 個特徵在所有樣本中的平均值，$\sigma_j$ 則是第 $j$ 個特徵的標準差。
$$
\begin{align}
\mu_j &= \frac{1}{m} \sum_{i=0}^{m-1} x^{(i)}_j \tag{5}\\
\sigma^2_j &= \frac{1}{m} \sum_{i=0}^{m-1} (x^{(i)}_j - \mu_j)^2  \tag{6}
\end{align}
$$

>**實作說明（Implementation Note）：** 在做特徵正規化時，必須記錄下用來正規化的那些數值──也就是每一個特徵的平均值與標準差。完成模型訓練並學得參數之後，我們通常會用它來預測訓練資料中沒出現過的房屋價格。對於新的輸入 x（例如客廳面積與臥室數），在帶入模型之前，必須先用「訓練資料計算出來的平均值與標準差」來對 x 做同樣的正規化。

**實作（Implementation）**

In [ ]:
def zscore_normalize_features(X):
    """
    computes  X, zcore normalized by column
    
    Args:
      X (ndarray (m,n))     : input data, m examples, n features
      
    Returns:
      X_norm (ndarray (m,n)): input normalized by column
      mu (ndarray (n,))     : mean of each feature
      sigma (ndarray (n,))  : standard deviation of each feature
    """
    # find the mean of each column/feature
    mu     = np.mean(X, axis=0)                 # mu will have shape (n,)
    # find the standard deviation of each column/feature
    sigma  = np.std(X, axis=0)                  # sigma will have shape (n,)
    # element-wise, subtract mu for that column from each example, divide by std for that column
    X_norm = (X - mu) / sigma      

    return (X_norm, mu, sigma)
 
#check our work
#from sklearn.preprocessing import scale
#scale(X_orig, axis=0, with_mean=True, with_std=True, copy=True)

接下來我們來看看 z-score 正規化的各個步驟。下圖會一步步顯示特徵在正規化前、中、後的變化。

In [ ]:
mu     = np.mean(X_train,axis=0)   
sigma  = np.std(X_train,axis=0) 
X_mean = (X_train - mu)
X_norm = (X_train - mu)/sigma      

fig,ax=plt.subplots(1, 3, figsize=(12, 3))
ax[0].scatter(X_train[:,0], X_train[:,3])
ax[0].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[0].set_title("unnormalized")
ax[0].axis('equal')

ax[1].scatter(X_mean[:,0], X_mean[:,3])
ax[1].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[1].set_title(r"X - $\mu$")
ax[1].axis('equal')

ax[2].scatter(X_norm[:,0], X_norm[:,3])
ax[2].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[2].set_title(r"Z-score normalized")
ax[2].axis('equal')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle("distribution of features before, during, after normalization")
plt.show()

上面的圖顯示訓練集中兩個參數「age」與「size(sqft)」之間的關係，且 *兩個軸使用相同尺度*：
- 左圖（Unnormalized）：尚未正規化時，`size(sqft)` 的數值範圍或變異遠大於 `age`。
- 中圖（X − μ）：第一步是從每個特徵中減去其平均值，讓特徵大致以 0 為中心。對 `age` 來說變化不太明顯，但可以明顯看到 `size(sqft)` 已經移到 0 附近。
- 右圖（Z-score normalized）：第二步再除以標準差，讓兩個特徵都以 0 為中心，且具有相近的尺度。

接著，我們將整個資料集做正規化，並與原始資料做比較。

In [ ]:
# normalize the original features
X_norm, X_mu, X_sigma = zscore_normalize_features(X_train)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma}")
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_train,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm,axis=0)}")

可以看到，每一個欄位（特徵）的「峰到峰範圍」（最大值減最小值）從原本相差上千的尺度，經過正規化之後，縮小到大約 2–3 的範圍。

In [ ]:
fig,ax=plt.subplots(1, 4, figsize=(12, 3))
for i in range(len(ax)):
    norm_plot(ax[i],X_train[:,i],)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("count");
fig.suptitle("distribution of features before normalization")
plt.show()
fig,ax=plt.subplots(1,4,figsize=(12,3))
for i in range(len(ax)):
    norm_plot(ax[i],X_norm[:,i],)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("count"); 
fig.suptitle("distribution of features after normalization")

plt.show()

請注意，上圖中正規化後的資料（x 軸）大致以 0 為中心，範圍約在 ±2 之間；更重要的是，各個特徵的範圍現在彼此之間相當接近。

接下來，我們使用正規化後的資料重新執行梯度下降演算法。
請注意，這次我們可以使用**大非常多的 alpha 值**，這會大幅加快梯度下降的收斂速度。

In [ ]:
w_norm, b_norm, hist = run_gradient_descent(X_norm, y_train, 1000, 1.0e-1, )

使用經過縮放的特徵後，可以在**短得多的時間內**獲得非常精確的結果！你會發現，在這段相對短的訓練過程結束時，每一個參數的梯度都已經非常小。對於使用正規化特徵的線性迴歸而言，學習率設為 0.1 是一個不錯的起點。

接著，我們將預測值與目標值畫在一起。請注意：模型實際做預測時用的是「正規化後的特徵」，但繪圖時 x 軸仍顯示「原始的特徵值」，方便解讀。

In [ ]:
#predict target using normalized features
m = X_norm.shape[0]
yp = np.zeros(m)
for i in range(m):
    yp[i] = np.dot(X_norm[i], w_norm) + b_norm

    # plot predictions and targets versus original features    
fig,ax=plt.subplots(1,4,figsize=(12, 3),sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train, label = 'target')
    ax[i].set_xlabel(X_features[i])
    ax[i].scatter(X_train[:,i],yp,color=dlc["dlorange"], label = 'predict')
ax[0].set_ylabel("Price"); ax[0].legend();
fig.suptitle("target versus prediction using z-score normalized model")
plt.show()

結果看起來相當不錯。這裡有幾點需要特別注意：
- 當特徵維度超過一個時，無法只用一張圖就把「結果 vs. 特徵」完整呈現出來。
- 在產生這些圖時，實際預測是使用「正規化後的特徵」計算出來的；凡是使用從正規化訓練集學得之參數所做的預測，其輸入特徵也都必須先做相同的正規化處理。

**預測（Prediction）**
我們建立這個模型的目的，就是要用它來預測資料集中「沒有出現過」的房屋價格。現在來預測一棟房子的價格，其條件為：1200 平方英尺、3 間臥室、1 層樓、屋齡 40 年。請記得，必須使用在訓練資料做正規化時所計算出的平均值與標準差，先對這筆輸入做正規化，再帶入模型進行預測。

In [ ]:
# First, normalize out example.
x_house = np.array([1200, 3, 1, 40])
x_house_norm = (x_house - X_mu) / X_sigma
print(x_house_norm)
x_house_predict = np.dot(x_house_norm, w_norm) + b_norm
print(f" predicted price of a house with 1200 sqft, 3 bedrooms, 1 floor, 40 years old = ${x_house_predict*1000:0.0f}")

**成本等高線（Cost Contours）**  

另一種理解特徵縮放的方法，是從「成本函數的等高線圖」來觀察。當各特徵的尺度不一致時，在參數空間中繪製的成本等高線會呈現非常不對稱的形狀。

在下圖中，參數的尺度已經過調整：左圖是在**未正規化特徵**的狀況下，以 w[0]（對應平方英尺）與 w[1]（對應臥室數）為軸畫出的成本等高線。由於尺度嚴重不對稱，等高線被拉得又長又扁，幾乎看不到完整的封閉曲線。相反地，在右圖中，當特徵經過正規化後，成本等高線就對稱得多。這代表在梯度下降的過程中，各個參數可以比較均衡地朝最小值前進，每個方向都能取得類似的進展。


In [ ]:
plt_equal_scale(X_train, X_norm, y_train)

<br>


## 恭喜！（Congratulations）
在本實驗中，你已經：
- 使用你在先前實驗中為「多特徵線性迴歸」所撰寫的各種函式
- 探索了學習率 $\alpha$ 對收斂行為的影響
- 了解了利用 z-score 正規化進行特徵縮放，能大幅加速梯度下降的收斂

<br>

## 感謝（Acknowledgments）
本實驗所使用的房價資料，是取自 Dean De Cock 為資料科學教育整理的 [Ames Housing 資料集](http://jse.amstat.org/v19n3/decock.pdf)。